In [23]:
# library imports
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
from spacy.tokens import DocBin
import json
from datetime import datetime
from tqdm import tqdm
import re
from pathlib import Path


In [24]:
version = "23"
train_spacy_file_name = "input/Hindistan_"+ version + "_train.spacy"
dev_spacy_file_name = "input/Hindistan_"+ version + "_dev.spacy"
keys_file_name = "data/keys_fasil_"+ version + ".csv"
product_file_name = "data/Hindistan "+ version + "-Yeni Anahtar Kelime-"+ version + ".fasıl.xlsx"
product_out_file_name = "data/products_"+ version + ".csv"
product_sheet_name = "Sayfa1"
train_data_out_file_name = "data/training_data_"+ version + ".txt"
test_data_out_file_name = "data/test_data_"+ version + ".txt"
product_column_name = "CONSIGNMENT_PRODUCT_DESCRIPTION"
result_file_name = "data/result_"+ version + ".xlsx"

In [25]:
training_data = {'TRAINING_DATA': []}

def createTrainingData(text, kw_list):
    results = []
    entities = []
    try:    
        for kw in tqdm(kw_list):
            search = re.finditer(kw, text, flags=re.IGNORECASE)      
            all_instances = [[m.start(),m.end()] for m in search] 
            if len(all_instances)>0:
                for i in all_instances:
                    start = i[0]
                    end = i[1]
                    entities.append((start, end, "PRODUCT"))  
            #else:
                #print("No pattern matches found. Keyword:", kw)    
    except:
        pass
    if len(entities)>0:
        results = [text, {"entities": entities}]
        training_data['TRAINING_DATA'].append(results)
        return

In [26]:

keys = pd.read_csv(keys_file_name)
product_df = pd.read_excel(product_file_name, sheet_name=product_sheet_name)
product_df['product'] = product_df[product_column_name].str.lower()
#product_df= product_df[['product']]
#product_df = pd.DataFrame(product_df['product'].unique(), columns=['product'])
product_df.to_csv(product_out_file_name, index=False)
keys.describe()

,keys
count,1522
unique,1522
top,bone meal
freq,1


In [27]:
msk = np.random.rand(len(keys)) < 0.8
train = keys[msk]
test=keys[~msk]

In [ ]:
for p in product_df['product']:
    #print ("product -->", p); 
    createTrainingData(p, train['keys'])

In [29]:
training_data['TRAINING_DATA']

[['bone meal  ', {'entities': [(0, 9, 'PRODUCT')]}],
 ['meat and bone meal  ',
  {'entities': [(9, 18, 'PRODUCT'), (0, 18, 'PRODUCT')]}],
 ['pet foods (buffalo offals unfit for huma n consumption) ',
  {'entities': [(11, 54, 'PRODUCT')]}],
 ['2940 bags fish meal (unfit for human con sumption)packed in pp bags each bag weig hing 50.130 kg gro',
  {'entities': [(10, 19, 'PRODUCT')]}],
 ['3780 bags fish meal (unfit for human con sumption)packed in pp bags (we intend to claim reward under',
  {'entities': [(10, 19, 'PRODUCT')]}],
 ['4200 bags fish meal(unfit for human cons umption)packed in pp bags (we intend to claim reward under ',
  {'entities': [(10, 19, 'PRODUCT')]}],
 ['5040 bags fish meal (unfit for human con sumption)packed in pp bags (we intend to claim reward under',
  {'entities': [(10, 19, 'PRODUCT')]}],
 ['actipal hp1-s1 shrimp powder hydrolysate ',
  {'entities': [(15, 40, 'PRODUCT')]}],
 ['dry krill meal ', {'entities': [(4, 14, 'PRODUCT')]}],
 ['fish meal  ', {'entities': [

In [30]:
import json
with open(train_data_out_file_name, 'w') as training_data_file:
     training_data_file.write(json.dumps(training_data))


In [31]:
#create test data
test_data = {'TEST_DATA': []}

def createTestData(text, kw_list):
    results = []
    entities = []
    try:    
        for kw in tqdm(kw_list):
            search = re.finditer(kw, text, flags=re.IGNORECASE)      
            all_instances = [[m.start(),m.end()] for m in search] 
            if len(all_instances)>0:
                for i in all_instances:
                    start = i[0]
                    end = i[1]
                    entities.append((start, end, "PRODUCT"))  
            #else:
                #print("No pattern matches found. Keyword:", kw)    
    except:
        pass
    if len(entities)>0:
        results = [text, {"entities": entities}]
        test_data['TEST_DATA'].append(results)
        return

In [ ]:
for p in product_df['product']:
    #print ("product -->", p); 
    createTestData(p, test['keys'])

In [33]:
import json
with open(test_data_out_file_name, 'w') as test_data_file:
     test_data_file.write(json.dumps(test_data))

In [34]:
# define our training data to TRAIN_DATA
TRAIN_DATA = training_data['TRAINING_DATA']

# create a blank model
nlp = spacy.blank('en')

def create_training_set(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []

        # create span objects
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract") 

            # skip if the character indices do not map to a valid span
            if span is None:
                print("Skipping entity.")
            else:
                ents.append(span)
                # handle erroneous entity annotations by removing them
                try:
                    doc.ents = ents
                except:
                    # print("BAD SPAN:", span, "\n")
                    ents.pop()
        doc.ents = ents

        # pack Doc objects into DocBin
        db.add(doc)
    return db

TRAIN_DATA_DOC = create_training_set(TRAIN_DATA)

# Export results 
TRAIN_DATA_DOC.to_disk(train_spacy_file_name)

 38%|███▊      | 921/2452 [00:00<00:00, 3031.33it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.


 63%|██████▎   | 1557/2452 [00:00<00:00, 3090.44it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.

 87%|████████▋ | 2134/2452 [00:00<00:00, 2930.76it/s]


Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping enti

100%|██████████| 2452/2452 [00:00<00:00, 2964.78it/s]


Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping enti

In [35]:
# define our test data to TEST_DATA
TEST_DATA = test_data['TEST_DATA']

# create a blank model
nlp = spacy.blank('en')

def create_dev_set(TEST_DATA):
    db = DocBin()
    for text, annot in tqdm(TEST_DATA):
        doc = nlp.make_doc(text)
        ents = []

        # create span objects
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract") 

            # skip if the character indices do not map to a valid span
            if span is None:
                print("Skipping entity.")
            else:
                ents.append(span)
                # handle erroneous entity annotations by removing them
                try:
                    doc.ents = ents
                except:
                    # print("BAD SPAN:", span, "\n")
                    ents.pop()
        doc.ents = ents

        # pack Doc objects into DocBin
        db.add(doc)
    return db

TEST_DATA_DOC = create_dev_set(TEST_DATA)

# Export results 
TEST_DATA_DOC.to_disk(dev_spacy_file_name)

 34%|███▍      | 929/2721 [00:00<00:00, 3092.41it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.


 68%|██████▊   | 1842/2721 [00:00<00:00, 2978.47it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.


100%|██████████| 2721/2721 [00:00<00:00, 3006.73it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.


In [ ]:
#!python -m spacy init fill-config base_config.cfg config.cfg
#dont forget to change the train and dev params
!python -m spacy init config config.cfg --lang en --pipeline ner -F

In [36]:
#!python -m spacy train config.cfg  --output .\output\
#!python -m spacy train config.cfg --output .\output\
#!python -m spacy train config.cfg --output ./output --paths.train ./input/Hindistan_06.spacy --paths.dev ./input/Hindistan_06.spacy

#!python -m spacy train config.cfg --paths.train ./input/Hindistan_06.spacy --paths.dev ./input/Hindistan_06.spacy --output ./output
#!python -m spacy train config.cfg --paths.train ./input/Hindistan_88_train.spacy --paths.dev ./input/Hindistan_88_dev.spacy --output ./output

#calisan komut
#!python -m spacy train config.cfg --output ./output

#run this powershell prompt then copy ouput dir here
!python -m spacy train config.cfg --output E:\\output

[+] Created output directory: E:\output
[i] Saving to output directory: E:\output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.50    3.28   12.07    1.90    0.03
  0     200         75.05   1884.48   14.00   18.18   11.38    0.14
  1     400         83.72    636.45    8.21   13.01    5.99    0.08
  2     600        110.01    525.30   12.38   17.05    9.72    0.12
  3     800        137.52    441.94   13.44   19.01   10.40    0.13
  5    1000        189.44    420.53    8.24   12.84    6.06    0.08
  6    1200        191.31    410.59   10.58   16.21    7.86    0.11
  9    1400        311.74    401.25   11.15   

[2022-06-21 15:58:40,568] [INFO] Set up nlp object from config
[2022-06-21 15:58:40,576] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-06-21 15:58:40,580] [INFO] Created vocabulary
[2022-06-21 15:58:40,580] [INFO] Finished initializing nlp object
[2022-06-21 15:58:42,500] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [ ]:
product_df.head()


In [ ]:
# load the trained model
nlp_output = spacy.load("output/model-last")
result_df = pd.DataFrame(columns=["ID", "desc", "product"])

columns = list(result_df)
data = []
for i in range(len(product_df)) :
    text = product_df.iloc[i, 2]
    id = df.iloc[i, 0]
    #print("-------------------------------")
    doc = nlp_output(text)
    # customize the label colors
    colors = {"PRODUCT": "linear-gradient(90deg, #E1D436, #F59710)"}
    options = {"ents": ["PRODUCT"], "colors": colors}

    
    displacy.render(doc, style="ent", options=options)
 
    
    # print out the identified entities
    for ent in doc.ents:
        if ent.label_ == "PRODUCT":
            #print(ent.text, ent.label_)
            #print(text, "------>", ent.text)
            values = [id, text, ent.text]
            zipped = zip(columns, values)
            a_dictionary = dict(zipped)
            data.append(a_dictionary)
result_df = result_df.append(data, True)
result_df.to_excel(result_file_name, index=False)

In [ ]:
# load the trained model
nlp_output = spacy.load("output/model-last")
result_df = pd.DataFrame(columns=["ID", "desc", "product"])

columns = list(result_df)
data = []

for text in product_df:
    #print("-------------------------------")
    doc = nlp_output(text)
    # customize the label colors
    colors = {"PRODUCT": "linear-gradient(90deg, #E1D436, #F59710)"}
    options = {"ents": ["PRODUCT"], "colors": colors}

    
    displacy.render(doc, style="ent", options=options)
 
    
    # print out the identified entities
    for ent in doc.ents:
        if ent.label_ == "PRODUCT":
            #print(ent.text, ent.label_)
            #print(text, "------>", ent.text)
            values = [text, ent.text]
            zipped = zip(columns, values)
            a_dictionary = dict(zipped)
            data.append(a_dictionary)
result_df = result_df.append(data, True)

In [ ]:
result_df.to_excel(result_file_name, index=False)